Created by: Paige O'Neil and Kaleigh West

## API Dataset

In [1]:
#install pymysql if not already done
!pip install pymysql

In [2]:
# Import libraries necessary for API
import os
import json
import pprint
import requests
import requests.exceptions
import pandas as pd

# List of possible numbers for object IDs:
possible_ids = [1, 2, 3, 56, 34, 76, 98, 45, 89, 100, 200, 234, 567, 789, 236, 800]

# Base URL
print("Please use the following link as the base url: https://collectionapi.metmuseum.org/public/collection/v1/objects/")
base_url = input("Please enter the provided base URL: ")
if base_url != 'https://collectionapi.metmuseum.org/public/collection/v1/objects/':
  base_url = input("Please enter the link provided above.")
print("")

# Ask user for 5 numbers to use as 5 object IDs
print("Please provide a few numbers to retreive info from the Met Museum API!")

#check to make sure if num is entered, only between 0-15
object_id1 = input("Please give a number between 0-15:")
if object_id1.isnumeric() == True:
  if not(int(object_id1) >=0 and int(object_id1)<=15):
    object_id1 = input("Please only enter a number between 0 and 15 or else the number is out of range:")
object_id2 = input("Please give a number between 0-15:")
if object_id2.isnumeric()== True:
  if not(int(object_id2) >=0 and int(object_id2)<=15):
    object_id2 = input("Please only enter a number between 0 and 15 or else the number is out of range:")
object_id3 = input("Please give another number between 0-15:")
if object_id3.isnumeric()== True:
  if not (int(object_id3) >=0 and int(object_id3)<=15):
    object_id3 = input("Please only enter a number between 0 and 15 or else the number is out of range:")
object_id4 = input("Please give another number between 0-15:")
if object_id4.isnumeric()== True:
  if not (int(object_id4) >=0 and int(object_id4)<=15):
    object_id4 = input("Please only enter a number between 0 and 15 or else the number is out of range:")
object_id5 = input("Please give a final number between 0-15:")
if object_id5.isnumeric()== True:
  if not (int(object_id5) >=0 and int(object_id5)<=15):
    object_id5 = input("Please only enter a number between 0 and 15 or else the number is out of range:")
print("")

#check to make sure only number is entered
if object_id1.isnumeric() == False:
  object_id1 = input("Please only enter a number, and make sure it is between 0-15:")
if object_id2.isnumeric()== False:
  object_id2 = input("Please only enter a number, and make sure it is between 0-15:")
if object_id3.isnumeric()== False:
  object_id3 = input("Please only enter a number, and make sure it is between 0-15:")
if object_id4.isnumeric()== False:
  object_id4 = input("Please only enter a number, and make sure it is between 0-15:")
if object_id5.isnumeric()== False:
  object_id5 = input("Please only enter a number, and make sure it is between 0-15:")
print("")

# Create object list, using user inputted numbers as indices for retrieving from possible ID list
object_list = [possible_ids[int(object_id1)], possible_ids[int(object_id2)], possible_ids[int(object_id3)],
               possible_ids[int(object_id4)], possible_ids[int(object_id5)]]

# Make empty list to store URLs
url_list = []

# Create URLs based on user input
for object_id in object_list:
    url_list.append(f'{base_url}{object_id}')


# Make sure all desired fields are included in chosen object IDs
for url in url_list:
    # Make GET requests
    response0 = requests.get(url)

    # Check the status code-- if object info is missing, remove it from further analysis
    if response0.status_code != 200:
        url_list.remove(url)
        print(f"Failed to retrieve data due to missing object information: {response0.status_code}. Object has been removed from further analysis.")

    # Parse the JSON response
    object_data0 = response0.json()


# Make dataframe with raw data
raw_df_concat = pd.DataFrame()

for url in url_list:
    # Make GET requests
    response1 = requests.get(url)

    # Check the status code
    if response1.status_code != 200:
        print(f"Failed to retrieve data: {response1.status_code}")

    # Parse the JSON response
    object_data1 = response1.json()

    # Create DataFrames for each url
    if url_list.index(url) == 0:
        raw_df0 = pd.json_normalize(object_data1)
    if url_list.index(url) == 1:
        raw_df1 = pd.json_normalize(object_data1)
    if url_list.index(url) == 2:
        raw_df2 = pd.json_normalize(object_data1)
    if url_list.index(url) == 3:
        raw_df3 = pd.json_normalize(object_data1)
    if url_list.index(url) == 4:
        raw_df4 = pd.json_normalize(object_data1)

# Create concatenated dataframe, combining all info from all objects
raw_df_concat = pd.concat([raw_df0, raw_df1, raw_df2, raw_df3, raw_df4], ignore_index = True)

# Print number of rows and columns
print("Number of rows: ", raw_df_concat.shape[0])
print("Number of columns: ", raw_df_concat.shape[1])
print("")

# Display dataframe
raw_df_concat

print("")
print("Cleaning up the data and peforming some operations...")
# Create empty data drame to store object ID, title, begin date, and end date
object_df = pd.DataFrame(columns=['Object ID', 'Title', 'Begin Date', 'End Date'])

for url in url_list:
    # Make GET requests
    response = requests.get(url)

    # Check the status code
    if response.status_code != 200:
        print(f"Failed to retrieve data: {response.status_code}")

    # Parse the JSON response
    object_data = response.json() # makes it a JSON object

    # Extract objectID, title, objectBeginDate, objectEndDate
    object_ID = object_data['objectID']
    name = object_data['title']
    begin_date = object_data['objectBeginDate']
    end_date = object_data['objectEndDate']

    # Add extracted data to dataframe
    object_df = pd.concat([object_df, pd.DataFrame({'Object ID': [object_ID],
                                      'Title': [name],
                                      'Begin Date': [begin_date],
                                      'End Date': [end_date]})],
                   ignore_index=True)

# Print number of rows and columns
print("")
print("Updating number of rows and columns...")
print("Number of rows: ", object_df.shape[0])
print("Number of columns: ", object_df.shape[1])

# Display dataframe
object_df

# Import necessary SQLite modules
import pymysql
import sqlite3
from sqlalchemy import create_engine

# Get the desired data type (CSV or SQLite) from user and account for difference in capitalization
print("")
desired_data_type = input("Please input desired output data type (CSV or SQLite): ")
desired_data_type = desired_data_type.lower()

# Make sure the user entered a valid data type. If not, ask them to enter a valid one
if desired_data_type != 'csv' and desired_data_type != 'sqlite':
  desired_data_type = input("Invalid data type. Please input CSV or SQLite: ")
  desired_data_type = desired_data_type.lower()
  #if enters invalid data type again, exit program and explain how to use program successfully next time
  if desired_data_type != 'csv' and desired_data_type != 'sqlite':
    print("")
    print('Exiting program due to invalid output selection.')
    print("If you wish to run the program again, only enter 'CSV' or 'SQLite' as your desired output when prompted.")

# If they choose CSV
if desired_data_type == "csv":
    print("")
    print("Creating CSV...")
    # Convert dataframe to CSV
    object_df.to_csv('met_museum.csv', index=False)
    print("CSV successfully created!")

# If they choose SQLite
elif desired_data_type == "sqlite":
    print("")
    print("Creating SQLite database...")
    # Create/connect to met_objects.db
    connection = sqlite3.connect('met_museum.db')

    # Write the DataFrame to an SQLite table
    object_df.to_sql('object_info', connection, if_exists='replace', index=False)

    # Close the connection
    connection.close()
    print("SQLite database successfully created!")

Please use the following link as the base url: https://collectionapi.metmuseum.org/public/collection/v1/objects/
Please enter the provided base URL: https://collectionapi.metmuseum.org/public/collection/v1/objects/

Please provide a few numbers to retreive info from the Met Museum API!
Please give a number between 0-15:0
Please give a number between 0-15:1
Please give another number between 0-15:2
Please give another number between 0-15:3
Please give a final number between 0-15:4


Number of rows:  5
Number of columns:  57


Cleaning up the data and peforming some operations...

Updating number of rows and columns...
Number of rows:  5
Number of columns:  4

Please input desired output data type (CSV or SQLite): csv

Creating CSV...
CSV successfully created!


### CSV dataset

In [3]:
import pandas as pd
import sys
print("Please download the Sabrina Carpenter Discography CSV and store it locally.")
print("It can be found for download at the following link: https://www.kaggle.com/datasets/delfinaoliva/sabrina-carpenter-discography")

#read CSV
sabrina_df = pd.read_csv('sabrina_carpenter_discography.csv', sep=';', header = 0, encoding='latin1')

#ask user for desired output and convert to all lowercase to resolve any capitalization issues
print("")
choice = input("Enter 'JSON' if you'd like your output as a JSON object, and enter 'SQLite' if you'd like your output as an SQL Database.")
choice = choice.lower()

#make sure choice is valid output
if choice != 'json' and choice!= 'sqlite':
  choice = input("Please enter only either 'SQLite' or 'JSON' as your preferred output: ")
  choice = choice.lower()
  if choice != 'json' and choice!= 'sqlite':
    print("")
    print('Exiting program due to invalid output selection.')
    print("If you wish to run the program again, only enter 'JSON' or 'SQLite' as your desired output when prompted.")

if choice == 'json' or choice == 'sqlite':
  #find num of rows and columns
  print("")
  print("Finding numbers of rows and columns...")
  print("Number of rows: ", sabrina_df.shape[0])
  print("Number of columns: ", sabrina_df.shape[1])

  #drop unnecessary columns and rename poor column names
  #(help with renaming columns in Pandas df: https://www.freecodecamp.org/news/how-to-rename-a-column-in-pandas/)
  print("")
  print("Cleaning up the data and peforming some operations...")
  sabrina_df.drop(['uri'], axis = 1, inplace=True)
  sabrina_df.drop(['speechiness'], axis = 1, inplace=True)
  sabrina_df.drop(['valence'], axis = 1, inplace=True)
  sabrina_df.rename(columns={"track_musical_genre": "genre", "videoclip_views": "video_views", "\u00ef\u00bb\u00bftrack_id": "track_id"}, inplace=True)

  #find num of rows and columns after data cleaning
  print("")
  print("Updating numbers of rows and columns...")
  print("Number of rows: ", sabrina_df.shape[0])
  print("Number of columns: ", sabrina_df.shape[1])
  print("")

  #store updated df in new csv
  print("Storing updated CSV to disk...")
  sabrina_df.to_csv('updated_sabrina_discography.csv', index=False)
  print("Stored!")
  print("")

  #create json object if user selected json output
  if choice == 'json':
    #convert df to JSON object
    print('JSON Object selected...')
    sabrina_json = sabrina_df.to_json(orient='records', indent=4)
    print("JSON object created under variable name 'sabrina_json'!")

  #create sqlite databse if user selected sqlite output
  if choice == 'sqlite':
    print('SQLite Database selected...')

    #import necessary sql packages
    import sqlite3
    from sqlalchemy import create_engine
    import json

    #create connection, database, and cursor
    connection = sqlite3.connect('sabrina.db')
    cursor = connection.cursor()

    #convert df from csv file into sql database
    sabrina_df.to_sql('sabrina', connection, if_exists = 'replace', index=False)

    #close connection
    connection.close()
    print("Database successfully created!")



Please download the Sabrina Carpenter Discography CSV and store it locally.
It can be found for download at the following link: https://www.kaggle.com/datasets/delfinaoliva/sabrina-carpenter-discography

Enter 'JSON' if you'd like your output as a JSON object, and enter 'SQLite' if you'd like your output as an SQL Database.json

Finding numbers of rows and columns...
Number of rows:  68
Number of columns:  25

Cleaning up the data and peforming some operations...

Updating numbers of rows and columns...
Number of rows:  68
Number of columns:  22

Storing updated CSV to disk...
Stored!

JSON Object selected...
JSON object created under variable name 'sabrina_json'!
